# Предсказание ошибок

Нерадивый ML-инженер Тензорослав обучал линейную регрессию на некотором датасете. Однако, он не был знаком с градиентными методами оптимизации, поэтому Тензорослав решил искать оптимум с помощью случайного поиска. Для этого он сгенерировал несколько случайных векторов весов и считал среднеквадратичную ошибку (MSE) для каждого вектора. Однако в какой-то момент Тензорослав случайно удалил выборку, на которой он считал MSE.

Помогите Тензорославу закончить начатое и предскажите значение MSE для тех векторов весов, которые он еще не успел обработать.

## О датасете

Вам предлагаются три файла:

- `train_weights.csv` – содержит значения весов в колонках W0–W9 (10-мерные векторы) и соответствующие значения MSE в колонке MSE
- `test_weights.csv` – содержит тестовые векторы весов (для которых нужно предсказать MSE)
- `example` – пример корректной посылки в контест

## Что нужно сделать

Необходимо предсказать колонку MSE для векторов из файла test_weights.csv и загрузить файл answers в формате, аналогичном примеру.

### Критерий оценки

Корень из средней квадратичной логарифмической ошибки (RMSLE).

Баллы рассчитываются по формуле: $100 * max(min((0.3 - RMSLE) / 0.1, 1), 0)$

Для максимального балла (100) необходимо достичь $RMSLE ≤ 0.2$.

### Требуемый формат вывода

Файл answers (без пробелов в названии) в формате JSON:

```json
[
    {
        "W0": 0.465,
        "W1": 0.582,
        "W2": 1.071,
        "W3": -1.889,
        "W4": -0.92,
        "W5": -1.084,
        "W6": -1.003,
        "W7": -0.766,
        "W8": -0.858,
        "W9": 0.665,
        "MSE": 40.593
    },
    ...
]
```

## Решение

### Посмотрим на данные

In [6]:
import pandas as pd

# Посмотрим что в файле тренировочного датасета
path = "../data/01/train_weights.csv"
df = pd.read_csv(path)
df.head()

,W0,W1,W2,W3,W4,W5,W6,W7,W8,W9,MSE
0,0.355,0.612,0.482,-1.220,-0.412,-0.745,-1.172,-1.760,0.479,1.180,69.113
1,-0.571,0.767,0.537,-0.904,-0.114,0.042,0.739,0.110,-1.743,1.831,67.790
2,0.389,-0.994,-1.537,-0.871,-0.351,-0.679,-0.475,-1.019,0.555,-1.048,32.273
3,0.539,0.679,-1.504,-0.472,0.909,0.158,0.866,1.084,-1.127,-0.436,18.747
4,0.289,-0.284,0.330,1.462,-0.205,0.213,-0.133,-2.064,-0.846,-0.223,62.239


In [5]:
df.describe()

,W0,W1,W2,W3,W4,W5,W6,W7,W8,W9,MSE
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000
mean,-0.051335,0.024363,-0.002192,-0.009831,-0.008766,0.016567,0.047995,0.007519,0.070722,0.029495,50.295404
std,0.978915,0.981791,1.010881,1.000155,0.994548,1.034104,1.018878,1.032102,0.995187,0.964162,21.651050
min,-3.141000,-3.679000,-3.411000,-2.751000,-3.115000,-3.309000,-3.320000,-3.375000,-3.172000,-2.941000,10.219000
25%,-0.675250,-0.597250,-0.696250,-0.715250,-0.668750,-0.688000,-0.639750,-0.674250,-0.564250,-0.621750,34.314750
50%,-0.038500,0.038500,-0.009000,-0.012000,0.023500,0.058500,0.013500,0.018500,0.065000,0.024000,47.262500
75%,0.623250,0.682250,0.686250,0.662000,0.664250,0.709250,0.737500,0.730750,0.744750,0.680000,62.307250
max,2.710000,3.750000,2.958000,2.942000,2.915000,3.344000,3.475000,2.987000,3.312000,3.497000,141.154000


- Все признаки W0–W9 — нормальные вещественные числа, примерно от −3 до +3
- Целевая переменная MSE – положительная, варьируется от ~10 до ~140

### Что нужно сделать

Построить модель, которая научится по 10 числам (W0–W9) оценивать, насколько высока будет ошибка (MSE) линейной регрессии с такими весами.

- MSE – положительная величина → обучить модель обычной регрессии
- Сделать предсказания для test_weights.csv
- Сохранить результат в JSON формате, как показано в примере

In [8]:
import pandas as pd
import numpy as np

# Читаем данные

train_path = "../data/01/train_weights.csv"
test_path = "../data/01/test_weights.csv"

train_df = pd.read_csv(train_path)
test_df = pd.read_csv(test_path)

X_train = train_df.drop(columns=["MSE"])
y_train = train_df["MSE"]
X_test = test_df.copy()

In [9]:
from sklearn.ensemble import RandomForestRegressor

# Обучаем модель

model = RandomForestRegressor(
    n_estimators=300,
    max_depth=12,
    random_state=42,
    n_jobs=-1
)
model.fit(X_train, y_train)

,n_estimators,300
,criterion,'squared_error'
,max_depth,12
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,1.0
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [10]:
# Делаем предсказания
y_pred = model.predict(X_test)

In [11]:
# Готовим итоговый df
results = X_test.copy()
results["MSE"] = y_pred

In [13]:
import json

# Сериализуем итоговый df в JSON

answers = results.to_dict(orient="records")

output_path = "answers.json"
with open(output_path, "w", encoding="utf-8") as f:
    json.dump(answers, f, ensure_ascii=False, indent=4)

print(f"Saved {len(answers)} predictions to {output_path}")

Saved 100 predictions to answers.json
